In [1]:
import os
import pickle
import numpy as np

from algos import *
from infect import infect
from sbm import SBM

In [2]:
import numpy as np

# binary spliting
def binary_splitting_round(s):
    # s: np.array the infectious status & test status
    num = 0
    flag = sum(s[:,0])>0
    assert flag
    stages = 0
    if len(s[:,0])==1:
        s[0,1] = s[0,0]
        return num,s,stages
    
    B1, B2 = np.array_split(s.copy(), 2,axis=0)
    flag = sum(B1[:,0])>0
    num+=1
    stages += 1
    
    if flag:
        n,stmp,stage = binary_splitting_round(B1)
        s[:len(B1),1] = stmp[:,1]
    else:
        s[:len(B1),1] = 0
        n,stmp,stage = binary_splitting_round(B2)
        s[len(B1):,1] = stmp[:,1]
    num += n
    stages += stage
    return num,s,stages 

def binary_splitting(s):
    # modified bs
    # s: 1-d array the infectious status
    st = np.zeros((len(s),2))
    st[:,0] = s
    st[:,1] = np.nan
    nums = 0
    count = sum(np.isnan(st[:,1]))
    stages = 0
    # the undetermined people
    while count!=0:
        mask = np.isnan(st[:,1])
        flag = sum(st[mask,0]>0)>0
        nums += 1
        stages+=1
        if not flag:
            st[mask,1] = 0
        else:
            n,stmp,stage = binary_splitting_round(st[mask,:])
            st[mask,1] = stmp[:,1]
            nums += n
            stages += stage
        count = sum(np.isnan(st[:,1]))
        
    assert sum(st[:,0]!=st[:,1])==0
    return nums,stages, st[:,1]

# diag
def diagalg_iter(s):
    # s(np.array): binary string of infection status
    k = int(np.log2(len(s)))
    l = int(2**(k-1))
    lp = 0
    p = np.zeros(k+1)
    group = dict()
    num = np.ones(k+1,dtype=np.int32)
    for i in range(k):
        p[i] = sum(s[lp:lp+l])>0
        group[i] = s[lp:lp+l]
        num[i] = l
        lp+=l
        l = l//2

    p[-1] = s[-1]
    group[k] = np.array([s[-1]])
    # p(array): pattern
    # group(dict): indicate the group information
    # num(array): the group size
    return p.astype(np.int32), group,num


def diag_splitting(s):
    # s(np.array): binary string of infection status
    num_tests = 0
    stages = 0
    pattern, group, nums = diagalg_iter(s)
    stages +=1
    num_tests += len(pattern)
    indices = np.where(pattern == 1)[0]
    flag = 0
    for i in indices:
        if nums[i]>1:
            num_test,stage = diag_splitting(group[i])
            num_tests += num_test
            if not flag:
                stages+=stage
                flag = 1
    return num_tests,stages

In [3]:
def num_infected(s):
    inf_people = 0
    for i in range(len(s)):
        inf_people += s[i]
    return inf_people

In [4]:
def Qtesting1(s):
    '''
    s(np.array): binary string of infection status
    '''
    num_tests = 0
    stages = 0
    ###################################################
    '''your code here'''
    print("RUNNING TEST ON original: ", s, "\n")
    num_tests, stages = mona(s, stages)

    ###################################################

    return num_tests, stages

def Qtesting2(s):
    '''
    s(np.array): binary string of infection status
    '''
    num_tests = 0
    stages = 0
    ###################################################
    '''your code here'''
    print("RUNNING TEST ON original: ", s, "\n")
    num_tests, stages = pablo(s, stages)
    ###################################################

    return num_tests,stages


def mona(s, max_stages):
    inf_people = num_infected(s)
    print("\nTESTING group: ", s)
  

    if (inf_people  == 1):
        binary_tests, binary_stages, _ = binary_splitting(s)
        print("--Num binary tests: ", binary_tests)
        print("--Num binary stages: ", binary_stages)
        return 1 + binary_tests, 1 + binary_stages
    elif(inf_people < 1):
        print("0 infected --> ran only 1 test")
        return 1, 1
    else:
        num_per_group = round(len(s)/inf_people)
        
        # for each subgroup, call function again
        tests = 1 # = this testing group
        for i in range(inf_people):
            stages = 1
            if(i == inf_people-1):
                subGroup = s[i*num_per_group:]
            else:
                subGroup = s[i*num_per_group:(i+1)*(num_per_group)]
            
            t, st = mona(subGroup, max_stages)
            stages += st
            tests += t   # number of tests in subtree

            print("Setting stages to ", max(max_stages, stages))
            max_stages = max(max_stages, stages)
        return tests, max_stages

#Returns a quantized number depending on infections
#0 
#1 (1<= I < 2)
#2 (2 <= I < 4)
#3 (4 <= I < 8)
#4 (>= 8)
def num_infected2(s):
    inf_people = 0
    for i in range(len(s)):
        inf_people += s[i]
    
    if(inf_people == 0):
        return 0
    elif(inf_people == 1):
        return 1
    elif(inf_people == 2 or inf_people == 3):
        return 2
    elif (inf_people < 8):
        return 3
    else:
        return 4
    
    
def pablo(s, max_stages):
    inf_range = num_infected2(s)
    print("\nTESTING group: ", s)
  
    if (inf_range == 0):
        print("0 infected --> ran only 1 test")
        return 1, 1
    elif (inf_range  == 1):
        # Only 1 infected person
        if len(s) == 1:
            return 1,1
        else:
            binary_tests, binary_stages, _ = binary_splitting(s)
            print("--Num binary tests: ", binary_tests)
            print("--Num binary stages: ", binary_stages)
            return 1 + binary_tests, 1 + binary_stages
   
    inf_people = 0
    if (inf_range == 2):
        inf_people = 3
    elif (inf_range == 3):
        inf_people = 7
    else:
        inf_people = 8

    num_per_group = round(len(s)/inf_people)
        
        # for each subgroup, call function again
    tests = 1 # = this testing group
    for i in range(inf_people):
        stages = 1
        if(i == inf_people-1):
            subGroup = s[i*num_per_group:]
        else:
            subGroup = s[i*num_per_group:(i+1)*(num_per_group)]
            
        t, st = pablo(subGroup, max_stages)
        stages += st
        tests += t   # number of tests in subtree

        # delete:
        m = max(max_stages, stages)
        if (m == stages and m != max_stages):
            print("Updating max stages from ", max_stages, " to ", stages)

        max_stages = max(max_stages, stages)
    return tests, max_stages

In [10]:
def infect_step(G,p1,individuals,N):
    '''The function serves as the infection model for each day.
    input params (consistent with the project description):

    
    G (ndarray N*N): the adjacency matrix.
    p1: the probability each individual infects neighbours.
    individuals = who's already infected
    N = # of people in 'individuals'
    '''

    ###################################################
    '''your code here'''
    individuals_updated = np.copy(individuals)
    for i in range(N):
        if individuals[i] == 1:     # this individual is infected
            ''' Need to find all of their neighbors and infect them w/
                probability p1 '''
            for neighbor_indx in range(N):
                if G[i][neighbor_indx] == 1:    # if i is neighbors w/ neighbor_indx

                    if individuals_updated[neighbor_indx] == 0: # if this person is NOT already infected
                        individuals_updated[neighbor_indx] = np.random.choice([0,1], p=[1-p1, p1])

                    # for debugging purposes:
                    #if individuals_updated[neighbor_indx] == 1:
                     #   print("\tInfecting ", neighbor_indx, ", who is a neighbor of ", i)
        
    ###################################################
    return individuals_updated



def infect(G,p0,p1,time_steps):
    '''The function serves as the infection model for each day.
    input params (consistent with the project description):
    G (ndarray N*N): the adjacency matrix.
    p0: the infection probability for initial status.
    p1: the probability each individual infects neighbours.
    time_steps: log N
    '''
    N = G.shape[0]
    individuals = np.zeros(N)
    ###################################################
    '''your code here'''
    for i in range(N):
        # infect everyone w/ initial probability p0
        individuals[i] = np.random.choice([0,1], p=[1-p0, p0])
    print("Original infected individuals: ", individuals)
    
    for _ in range(time_steps):
        individuals = infect_step(G, p1, individuals, N)
        print("Infected individuals after step ", _, " - ", individuals)
        
    ###################################################

    return individuals

In [ ]:
''' 
Idea:
- select R representatives per family
- test that group of representatives
- if POS:
    individually test

    ASK - do we want to individually, test? what if R = 100 and the test outputs 1 infected person - should we run our algo on this also

if NEG:
    group test whole family
    if NEG --> done
    if POS --> run our original Qtesting1 and Qtesting 2

How to reorganize 'leaves'

communities[i] = index of community that person i is in
'''

In [7]:
def create_communities(N, M):
    size = round(N/M)
    communities = np.zeros(N)
    
    for i in range(N):
        i_subgroup = np.floor((i)/size)       # which subgroup this index is in
        communities[i] = i_subgroup
    return communities

In [135]:
R = 5

In [148]:
def test_communities(s, communities):
    tests = 1

    com_results = []
        # For community i, com_results[i][0] = how many of the R representatives were infected
        #                  com_results[i][1] = total # infected in community i


    com_group = 0
    inf_people = 0
    representative_count = 0
    person = 0
    temp = [0,0]
    while person < len(communities):
        curr_com = communities[person]

        # If this person is in a new community
        if (curr_com != com_group):
            #print("----------Total infected in ", com_group, ": ", inf_people)
            com_results.append(temp)
            #print("Adding ", temp, " to COM_RESULTS ", com_group)
            temp = [0,0]
            com_group = curr_com
            tests += 1

            #print("\nReached community ", com_group, " at index ", person)
            inf_people = 0
        
        # Once you've tested R representatives in this family
        if (representative_count == R):
            representative_count = 0
            temp[0] = inf_people

            # Skip all the rest of the people in this community
            while (person < len(communities) and communities[person] == com_group):
                inf_people += s[person]
                person += 1
            temp[1] = inf_people

        # Otherwise, count # of infected people w/in R representatives of this community
        else:
            inf_people += s[person]     
            representative_count += 1
            #print("Examining person ", person, " in comm ", com_group)
            person += 1


    #print("----Total infected in ", com_group, ": ", inf_people)
    com_results.append(temp)
    #print("Adding ", temp, " to COM_RESULTS ", com_group)

    return tests, com_results

In [153]:
def comm(s, communities):
    tests, results = test_communities(s, communities)
    print("\n*******************************\nInitial round of testing used ", tests, " tests")

    print(results)

    for c in range(len(results)):
        num_reps = results[c][0]
        num_total = results[c][1]
        print("\nCOMM ", c, " -- Found ", num_reps, " infected representatives and ", num_total, " infected in total")
        if (num_reps == 0):
            print("COM ", c, " is likely NOT infected")

In [155]:
N = 50
M = 5
q0 = .4
q1 = 0

p0 = .1
p1 = .5
time_steps = 2

comms = create_communities(N, M)
print("COMMUNITIES: ", comms, "\n")
G = SBM(N, M, q0,q1)
#print(G)

s = infect(G, p0, p1, time_steps)
comm(s, comms)

COMMUNITIES:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 2. 2. 2. 2.
 2. 2. 2. 2. 2. 2. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 4. 4. 4. 4. 4. 4. 4. 4.
 4. 4.] 

Original infected individuals:  [0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
Infected individuals after step  0  -  [1. 1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
Infected individuals after step  1  -  [1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]

*******************************
Initial round of testing used  5  tests
[[3.0, 7.0], [1.0, 2.0], [1.0, 1.0], [0.0, 0.0], [0.0, 0.0]]

COMM  0  -- Found  3.0  infected representatives and  7.0  infected in total

COMM  1  -- Found  1.0  infected representati